In [ ]:
# Importing essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns # رسم
sns.set() # رسم
import warnings # اخفا الرسايل الغيرر مهمه
warnings.filterwarnings('ignore')

In [ ]:
# Loading the datasets into Pandas DataFrames
departments = pd.read_csv('departments.csv') #
order_products_train = pd.read_csv('/content/order_products__train.csv')
order_products_prior = pd.read_csv('/content/order_products__prior.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv', on_bad_lines='skip')
aisles = pd.read_csv('aisles.csv')

In [ ]:
# Displaying the first five rows of the 'departments' DataFrame
departments.head() # عرضنا اول خمس صفوف

In [ ]:
plt.figure(figsize=(12,6))

sns.barplot(x = departments['department'].value_counts()[:10].index,
           y = departments['department'].value_counts()[:10].values)
plt.xticks(size=12, rotation=45)
plt.title('10 Best department')
plt.show() # افضل عشر اقسام

In [ ]:
# Displaying number of rows and col of the 'departments' DataFrame
departments.shape

In [ ]:
# Displaying the first five rows of the 'order_product' DataFrame
order_products_train.head()
# رقم الطلب \ رقم المننتج \ اعادة الطلب

In [ ]:
# Displaying the number of col and rows of the 'order_product' DataFrame
order_products_train.shape

In [ ]:
# Displaying the number of cols and row of the 'order_product' DataFrame
order_products_prior.head()
# هذا الجدول قبل التدريب

In [ ]:
# count of eval_set
orders["eval_set"].value_counts() # نتيجة تحتوي على كل نوع مجموعة تقييم وعدد مرات ظهوره في البيانات
#

In [ ]:
products.head()
# aisle هو رقم الممر
# يعني مثلا رقم الممر في قسم الخضروات

In [ ]:
# merge data
dataset_train = orders[orders['eval_set'] == 'train'].merge(order_products_train , on = 'order_id')
dataset_train
# دمج الداتا سيت

In [ ]:
dataset_train.isnull().sum()/len(dataset_train) * 100
# ياخذ القيم المفقودة في كل عامود ويقسمها على عدد الصفوف الكلي
# جميعها لا يوجد فيها اصفار الا اخر شي فيها نل

In [ ]:
dataset_test = orders[orders['eval_set'] == 'test']
dataset_test

In [ ]:
# merge the data
prior  = orders.merge(order_products_prior , on = 'order_id')
prior#سيحتوي على معلومات الطلبات السابقة (prior orders) مع تفاصيل المنتجات المطلوبة في كل طلب

In [ ]:
# miss data
prior.isnull().sum()/len(prior) * 100

In [ ]:
prior["days_since_prior_order"] = prior["days_since_prior_order"].fillna(prior["days_since_prior_order"].median() ) # يشيل القيم النل ويحط المتوسط الحسابي

In [ ]:
prior["no of time buy"] = prior.groupby("user_id")['product_id'].count()# يسوي مجموعه بنا على البرودكت اي دي واليوزر اي دي

In [ ]:
prior["no of"] = prior.groupby("product_id")['user_id'].count() # يحسب عدد القروب اللي فيه البرودكت واليوزر اي دي

In [ ]:
prior

In [ ]:
dataset_test = dataset_test.drop(['order_id','user_id','eval_set'] , axis = 1) # لانها غير ضروريه

In [ ]:
# spilt data
x = dataset_train.drop(['order_id','user_id','eval_set', 'product_id' , 'add_to_cart_order' , 'reordered' ] , axis = 1)
y = dataset_train['reordered']

In [ ]:
y.value_counts()/len(y)*100

In [ ]:
#loading the library
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = variance_inflation_factor
data = pd.DataFrame({'Feature names' : x.columns , 'vif' : [vif(x.values ,i) for i in range(x.shape[1])]})
data

In [ ]:
# find outlier
def outlier(col):
    q1 = dataset_test[col].quantile(0.25)
    q3 = dataset_test[col].quantile(0.75)
    iqr = q3 - q1

    lower = q1-1.5*iqr# اي قيمه اقل من هذا الحد تعتبر شاذه
    upper = q1 + 1.5*iqr # اي قيمه اقل من هذا الحد تعتبر شاذه

    dataset_test[col] = np.where(dataset_test[col]< lower , lower , np.where(dataset_test[col] > upper , upper , dataset_test[col]))

# لتحل محل أي قيمة أقل من الحد الأدنى بقيمة الحد الأدنى نفسه.

for col in dataset_test.columns:
    outlier(col)
    # يمر على جميع الاعمده ويستخدم داله الاوتلير

In [ ]:
dataset_test

In [ ]:
x_train = x
y_train = y
x_test = dataset_test

In [ ]:
# model logistic
import numpy as np
from sklearn.linear_model import LogisticRegression # Import LogisticRegression

# Remove rows where y_train is NaN
mask = ~np.isnan(y_train)
x_train_clean = x_train[mask]
y_train_clean = y_train[mask]

lr = LogisticRegression()
lr.fit(x_train_clean, y_train_clean)

In [ ]:
# model logistic
import numpy as np
from sklearn.linear_model import LogisticRegression # Import LogisticRegression
from sklearn import metrics # Import the metrics module

# Remove rows where y_train is NaN
mask = ~np.isnan(y_train)
x_train_clean = x_train[mask]
y_train_clean = y_train[mask]

lr = LogisticRegression()
lr.fit(x_train_clean, y_train_clean)

# Predict using the cleaned data
y_pred_train = lr.predict(x_train_clean)
y_pred_test = lr.predict(x_test)

# Train accuracy
print("*" * 40)
print("Train accuracy:", metrics.accuracy_score(y_train_clean, y_pred_train)) # Now metrics is defined
print("*" * 40)

# Train precision_score
print("Train precision_score:", metrics.precision_score(y_train_clean, y_pred_train, average='weighted'))
print("*" * 40)

# Train recall_score
print("Train recall_score:", metrics.recall_score(y_train_clean, y_pred_train, average='weighted'))
print("*" * 40)

# Train f1_score
print("Train f1_score:", metrics.f1_score(y_train_clean, y_pred_train, average='weighted'))
print("*" * 40)

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules # داله تستخدم لاستخراج الانماط المتكرره ، لتكتشف العلاقات بين العتاصر  المختلفه

In [ ]:
basket = dataset_test.groupby(['order_number', 'order_dow'])['order_dow'].count().unstack()  .fillna(0)
 #يسوي جدول
basket[basket > 0] = 1 #  يقرب الى باينري ، واحد طلب في هذا اليوم اما صفر لم يطلب

In [ ]:
frequent_itemsets = apriori(basket, min_support=0.1, use_colnames=True)

# Generate association rules from frequent itemsets
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1.0)

# Display the resulting association rules
print(rules)